Installlation

In [1]:
%%bash
pip install pytorch-transformers

cd /home/ec2-user/SageMaker/pytorch-transformers/examples
pip install -r requirements.txt


#Fixing error with from tensorboardX import SummaryWriter
#TypeError: __new__() got an unexpected keyword argument 'serialized_options'

source activate pytorch_p36
pip install protobuf -U



Requirement already up-to-date: protobuf in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (3.9.1)
Requirement not upgraded as not directly required: setuptools in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (from protobuf) (39.1.0)
Requirement not upgraded as not directly required: six>=1.9 in /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (from protobuf) (1.11.0)


fastai 1.0.55 requires nvidia-ml-py3, which is not installed.
thinc 6.12.1 has requirement msgpack<0.6.0,>=0.5.6, but you'll have msgpack 0.6.0 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
fastai 1.0.55 requires nvidia-ml-py3, which is not installed.
thinc 6.12.1 has requirement msgpack<0.6.0,>=0.5.6, but you'll have msgpack 0.6.0 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
fastai 1.0.55 requires nvidia-ml-py3, which is not installed.
thinc 6.12.1 has requirement msgpack<0.6.0,>=0.5.6, but you'll have msgpack 0.6.0 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Download and Generate Data suitable with run_glue example

In [2]:
import boto3

s3 = boto3.client('s3')
bucket_name = 'ipa-cognitive-futures'
object_name = 'cf_datasets/TM/raw/nc020-a04_ibli.csv'
file_name = 'nc020-a04_ibli.csv'
semantic_column = 'EN'

s3.download_file(bucket_name, object_name, file_name)

import pandas as pd
nice_df = pd.read_csv(file_name, header=0,encoding='latin1')

#correct columns
retrieved_columns = ['Class','Classification','EN', 'FR']
columns = retrieved_columns + list(nice_df.columns)[4:]

#obtain EN GS
nice_df.columns = columns
nice_df = nice_df[retrieved_columns]
del nice_df['FR']
nice_df.dropna(inplace=True)
nice_df['Class'] = nice_df['Class'].astype(int)
nice_df['Classification'] = nice_df['Classification'].astype(int)
nice_df.reset_index(inplace=True,drop=True)
nice_df.head()

,Class,Classification,EN
0,1,10001,combusting preparations [chemical additives to...
1,1,10002,adhesives for industrial purposes
2,1,10003,"salt for preserving, other than for foodstuffs"
3,1,10004,auxiliary fluids for use with abrasives
4,1,10005,vulcanization accelerators


In [3]:
d = nice_df.groupby('Class').head(1)
del d['Classification']
d.to_csv('/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets/gns/def.tsv',sep='\t',header=False,index=False)

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
train, dev = train_test_split(nice_df, train_size=0.8, random_state=7, stratify=list(nice_df['Class']))

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
nice_df.groupby('Class').count().head(10)

,Classification,EN
Class,,
1,752,752
2,132,132
3,272,272
4,110,110
5,525,525
6,506,506
7,613,613
8,300,300
9,832,832


In [7]:
train.groupby('Class').count().head(10)

,Classification,EN
Class,,
1,602,602
2,106,106
3,218,218
4,88,88
5,420,420
6,405,405
7,490,490
8,240,240
9,666,666


In [8]:
dev.groupby('Class').count().head(10)

,Classification,EN
Class,,
1,150,150
2,26,26
3,54,54
4,22,22
5,105,105
6,101,101
7,123,123
8,60,60
9,166,166


In [9]:
dataset_folder = '/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets/gns'
#train.to_csv(dataset_folder + '/train.tsv', sep='\t', index=False, header=False)
#dev.to_csv(dataset_folder + '/dev.tsv', sep='\t', index=False, header=False)

In [10]:
train = pd.read_csv(dataset_folder + '/train.tsv', sep='\t',  header=None)
train.columns = ['Class','Classification','En']
dev = pd.read_csv(dataset_folder + '/dev.tsv', sep='\t',  header=None)
dev.columns = ['Class','Classification','En']
dev.head()


,Class,Classification,En
0,20,200235,stools
1,3,30082,diamantine [abrasive]
2,25,250095,cuffs
3,22,220071,tents*
4,11,110135,purification installations for sewage


### Run Glue Training on GnS

# For Training V2 (bert-large)

In [ ]:
%%sh

export GLUE_DIR=/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets
export TASK_NAME=gns

python /home/ec2-user/SageMaker/pytorch-transformers/examples/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-large-uncased \
    --task_name $TASK_NAME \
    --do_train \
    --do_eval \
    --do_lower_case \
    --data_dir $GLUE_DIR/$TASK_NAME \
    --max_seq_length 15 \
    --per_gpu_eval_batch_size=128   \
    --per_gpu_train_batch_size=128   \
    --learning_rate 2e-5 \
    --num_train_epochs 100.0 \
    --output_dir $GLUE_DIR/output/$TASK_NAME/bert-large/ 2>&1 | tee run_bert_large_uncased.log


## Evaluate

In [19]:
%%sh

export GLUE_DIR=/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets
export TASK_NAME=gns

python /home/ec2-user/SageMaker/pytorch-transformers/examples/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-large-uncased \
    --task_name $TASK_NAME \
    --do_eval \
    --do_lower_case \
    --data_dir $GLUE_DIR/$TASK_NAME \
    --max_seq_length 15 \
    --per_gpu_eval_batch_size=128   \
    --per_gpu_train_batch_size=128   \
    --learning_rate 2e-5 \
    --num_train_epochs 100.0 \
    --output_dir $GLUE_DIR/output/$TASK_NAME/bert-large/ 2>&1 | tee run_bert_large_uncased.log


09/19/2019 23:52:28 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
09/19/2019 23:52:29 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmp_5rdzouq
100%|██████████| 314/314 [00:00<00:00, 114056.59B/s]
09/19/2019 23:52:30 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmp_5rdzouq to cache at /home/ec2-user/.cache/torch/pytorch_transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.4c88e2dec8f8b017f319f6db2b157fee632c0860d9422e4851bd0d6999f9ce38
09/19/2019 23:52:30 - INFO - pytorch_transformers.file_utils -   creating metadata file for /home/ec2-user/.cache/torch/pytorch_transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.4c88e2dec8f8b017f319f6db2b157fee632c0860d9422e4851bd0d6999f9ce38
09/19/2019 23:52:30 - INFO

## Predict from CMD

In [27]:
%%sh

export GLUE_DIR=/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets
export TASK_NAME=gns
export TEST_FILE=/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets/gns/test.tsv

python /home/ec2-user/SageMaker/pytorch-transformers/examples/run_glue_serving_class.py \
    --model_type bert \
    --model_name_or_path bert-large-uncased \
    --task_name $TASK_NAME \
    --do_test \
    --test_file $TEST_FILE \
    --do_lower_case \
    --data_dir $GLUE_DIR/$TASK_NAME \
    --max_seq_length 15 \
    --per_gpu_eval_batch_size=128   \
    --per_gpu_train_batch_size=128   \
    --learning_rate 2e-5 \
    --num_train_epochs 100.0 \
    --output_dir $GLUE_DIR/output/$TASK_NAME/bert-large/ 2>&1 | tee run_bert_large_uncased_predict.log


09/23/2019 12:27:18 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
09/23/2019 12:27:18 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at /home/ec2-user/.cache/torch/pytorch_transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.4c88e2dec8f8b017f319f6db2b157fee632c0860d9422e4851bd0d6999f9ce38
09/23/2019 12:27:18 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "gns",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 45,
  "output_attentions": false,
  "output_hidden_states": false,
  "

## Predict from object

In [161]:
%%time

import sys
sys.path.append('/home/ec2-user/SageMaker/pytorch-transformers/examples')


import utils_glue
import run_glue_serving_class

import imp
imp.reload(utils_glue)

imp.reload(run_glue_serving_class)
from run_glue_serving_class import PyTorchTransformer

from argparse import Namespace
args = Namespace()
args.model_type='bert'
args.task_name='gns'
args.do_test=True
args.test_file='/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets/gns/test.tsv'
args.do_lower_case=True
args.data_dir = '/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets/gns'
args.max_seq_length=15
args.per_gpu_eval_batch_size=128
args.per_gpu_train_batch_size=128
args.output_dir='/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets/output/gns/bert-large'
args.local_rank=-1 #default param that needs to be avaialble all the time
args.seed = 42
pt = PyTorchTransformer(args)

09/23/2019 14:23:46 - INFO - pytorch_transformers.tokenization_utils -   Model name '/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets/output/gns/bert-large' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc). Assuming '/home/ec2-user/SageMaker/pytorch-transformers/examples/datasets/output/gns/bert-large' is a path or url to a directory containing tokenizer files.
09/23/2019 14:23:46 - INFO - pytorch_transformers.tokenization_utils -   loading file /home/ec2-user/SageMaker/pytorch-transformers/examples/datasets/output/gns/bert-large/vocab.txt
09/23/2019 14:23:46 - INFO - pytorch_transformers.toke

CPU times: user 8.41 s, sys: 895 ms, total: 9.3 s
Wall time: 9.16 s


In [162]:
%%time
dataframe = pd.DataFrame({'Class':['1','3'],'Classification':['10001','30495'],'EN':['electric','coils, electric']})
result = pt.predict(dataframe=dataframe)

09/23/2019 14:23:55 - WARNING - utils_glue -   Unable to load definition
09/23/2019 14:23:55 - INFO - run_glue_serving_class -   Creating features from dataframe size=2
09/23/2019 14:23:55 - INFO - run_glue_serving_class -   label_list=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45']
09/23/2019 14:23:55 - INFO - utils_glue -   [before] list_of_lines=[['1', '10001', 'electric'], ['3', '30495', 'coils, electric']]
09/23/2019 14:23:55 - INFO - utils_glue -   [after] list_of_lines=[['1', '10001', 'electric'], ['3', '30495', 'coils, electric']]
09/23/2019 14:23:55 - INFO - run_glue_serving_class -   Examples=[<utils_glue.InputExample object at 0x7fdf8bba25c0>, <utils_glue.InputExample object at 0x7fdf8bba2358>]
09/23/2019 14:23:55 - INFO - utils_glue -   Writing example 0 of 2
09/2

label_defs_dict={}


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  3.32it/s]
09/23/2019 14:23:55 - INFO - run_glue_serving_class -   ***** Predict results  *****
09/23/2019 14:23:55 - INFO - run_glue_serving_class -   [('7', 5.868683, ''), ('11', 5.523337, ''), ('9', 2.8638172, ''), ('37', 1.535211, ''), ('4', 1.3527367, ''), ('12', 0.95695263, ''), ('13', 0.7478874, ''), ('17', 0.20180409, ''), ('27', 0.19748937, ''), ('38', 0.18053524, '')]
09/23/2019 14:23:55 - INFO - run_glue_serving_class -   [('9', 10.3631525, ''), ('11', 0.86887646, ''), ('13', 0.5492677, ''), ('38', 0.3931975, ''), ('1', 0.35635006, ''), ('24', 0.27513748, ''), ('12', 0.27274722, ''), ('7', 0.24356386, ''), ('6', 0.22364482, ''), ('10', 0.2159835, '')]
09/23/2019 14:23:55 - INFO - run_glue_serving_class -   Results=[[('7', 5.868683, ''), ('11', 5.523337, ''), ('9', 2.8638172, ''), ('37', 1.535211, ''), ('4', 1.3527367, ''), ('12', 0.95695263, ''), ('13', 0.7478874, ''), ('17', 0.20180409, ''), ('27', 0.19748937, ''), ('38', 0.180

probs=[[-0.952225   -0.78217405 -1.7648941   1.3527367  -1.0146585  -0.05956405
   5.868683   -1.3725636   2.8638172  -0.24768285  5.523337    0.95695263
   0.7478874  -0.50906235 -0.37397382 -0.87363523  0.20180409 -1.2136983
   0.08560596 -0.88931155 -0.5777061  -0.5987683  -0.35561106 -0.78652936
   0.17457873 -0.56643623  0.19748937 -0.3483013   0.08475477 -0.0308211
  -0.23981859 -0.76675665 -0.3522347  -0.24363081 -1.2640858  -0.7922492
   1.535211    0.18053524  0.14754567 -0.4648969  -0.7059946  -0.8118435
  -0.32292196 -0.95232487 -0.97321194]
 [ 0.35635006 -0.48779127 -0.5643626  -0.5515348  -0.12894934  0.22364482
   0.24356386 -0.5262091  10.3631525   0.2159835   0.86887646  0.27274722
   0.5492677   0.13683408 -0.17143418 -0.01728117  0.05178845 -0.5998312
  -1.8530825   0.10292944 -0.31142485 -1.1790127  -0.8000403   0.27513748
   0.06714713 -0.02486451 -0.75736344 -0.04116625 -1.0710574  -1.4890515
  -0.37593347 -1.1213502  -1.6555772  -0.8967663  -0.5864784  -1.3021399


In [164]:
result

[[('7', 5.868683, ''),
  ('11', 5.523337, ''),
  ('9', 2.8638172, ''),
  ('37', 1.535211, ''),
  ('4', 1.3527367, ''),
  ('12', 0.95695263, ''),
  ('13', 0.7478874, ''),
  ('17', 0.20180409, ''),
  ('27', 0.19748937, ''),
  ('38', 0.18053524, '')],
 [('9', 10.3631525, ''),
  ('11', 0.86887646, ''),
  ('13', 0.5492677, ''),
  ('38', 0.3931975, ''),
  ('1', 0.35635006, ''),
  ('24', 0.27513748, ''),
  ('12', 0.27274722, ''),
  ('7', 0.24356386, ''),
  ('6', 0.22364482, ''),
  ('10', 0.2159835, '')]]